# Rock Paper Scissors AI
This is the code for the code challenge **"Rock Paper Scissors"** for the FreeCodeCamp Machine Learning with Python ceritifcation.

In [1]:
import random
from collections import Counter

## RPS_game.py

In [2]:
def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (
            (p1_play == "P" and p2_play == "R")
            or (p1_play == "R" and p2_play == "S")
            or (p1_play == "S" and p2_play == "P")
        ):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif (
            p2_play == "P"
            and p1_play == "R"
            or p2_play == "R"
            and p1_play == "S"
            or p2_play == "S"
            and p1_play == "P"
        ):
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results["p2"] + results["p1"]

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results["p1"] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return win_rate

First opponent: **quincy**
Quincy is easy, since he basically uses the same combination all the times, so basically use the opposed sequence to win.

In [3]:
def quincy(prev_play, counter=[0]):
    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]

Second opponent: **mrugesh**
MRugesh takes a different approach... He reads the last 10 plays and throws the opposite of the most played hand.

In [4]:
def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == "":
        most_frequent = "S"

    ideal_response = {"P": "S", "R": "P", "S": "R"}
    return ideal_response[most_frequent]

Third opponent: **kris**
This is another easy one. He checks your last hand and will play the opposite. Just keep cycling your hand. If you previously played "Rock", play "Scissorr" next.

In [5]:
def kris(prev_opponent_play):
    if prev_opponent_play == "":
        prev_opponent_play = "R"
    ideal_response = {"P": "S", "R": "P", "S": "R"}
    return ideal_response[prev_opponent_play]

Last opponent: **Abbey**
Here's were the problem gets harder... Abbey will read your last two hands and will try to forecast your next action. In order to deafet her, we need to take the same approach, but considering more plays than 2.

In [6]:
def abbey(
    prev_opponent_play,
    opponent_history=[],
    play_order=[
        {
            "RR": 0,
            "RP": 0,
            "RS": 0,
            "PR": 0,
            "PP": 0,
            "PS": 0,
            "SR": 0,
            "SP": 0,
            "SS": 0,
        }
    ],
):
    if not prev_opponent_play:
        prev_opponent_play = "R"
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k] for k in potential_plays if k in play_order[0]
    }
    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {"P": "S", "R": "P", "S": "R"}
    return ideal_response[prediction]

The function to play agains a human.

In [7]:
def human(prev_opponent_play):
    play = ""
    while play not in ["R", "P", "S"]:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play

This is a funtion to play randomly, but we don't achieve the 60% of winning rate as expected...

In [8]:
def random_player(prev_opponent_play):
    return random.choice(["R", "P", "S"])

## Solution

In [9]:
# Here we keep track of previous plays
steps = {}

def player(prev_play, opponent_history=[]):
    if prev_play != "":
        opponent_history.append(prev_play)
    # Like previously stated, basiclly we'll follow the same strategy as Abbey, where "n" is how far we'll look.
    # If we go too far, it may not be too effective,.
    n = 5

    guess = "R"
    if len(opponent_history) > n:
        pattern = "".join(opponent_history[-n:])

        if "".join(opponent_history[-(n + 1):]) in steps.keys():
            steps["".join(opponent_history[-(n + 1):])] += 1
        else:
            steps["".join(opponent_history[-(n + 1):])] = 1

        possible = [pattern + "R", pattern + "P", pattern + "S"]

        for i in possible:
            if not i in steps.keys():
                steps[i] = 0

        predict = max(possible, key=lambda key: steps[key])

        if predict[-1] == "P":
            guess = "S"
        if predict[-1] == "R":
            guess = "P"
        if predict[-1] == "S":
            guess = "R"

    return guess

### Now, let's train our AI witth 1000 plays for each player

In [10]:
print("Training against Quincy:")
play(player, quincy, 1000)
print ("Training against MRugesh:")
play(player, mrugesh, 1000)
print("Training against Abbey:")
play(player, abbey, 1000)
print("Training against Kris:")
play(player, kris, 1000)

Training against Quincy:
Final results: {'p1': 992, 'p2': 3, 'tie': 5}
Player 1 win rate: 99.69849246231156%
Training against MRugesh:
Final results: {'p1': 828, 'p2': 165, 'tie': 7}
Player 1 win rate: 83.38368580060424%
Training against Abbey:
Final results: {'p1': 447, 'p2': 305, 'tie': 248}
Player 1 win rate: 59.441489361702125%
Training against Kris:
Final results: {'p1': 754, 'p2': 120, 'tie': 126}
Player 1 win rate: 86.2700228832952%


86.2700228832952

## Test sequence

In [11]:
import unittest

class UnitTests(unittest.TestCase):
    print()

    def test_player_vs_quincy(self):
        print("Testing game against quincy...")
        actual = play(player, quincy, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat quincy at least 60% of the time.')

    def test_player_vs_abbey(self):
        print("Testing game against abbey...")
        actual = play(player, abbey, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat abbey at least 60% of the time.')

    def test_player_vs_kris(self):
        print("Testing game against kris...")
        actual = play(player, kris, 1000) >= 60
        self.assertTrue(
            actual, 'Expected player to defeat kris at least 60% of the time.')

    def test_player_vs_mrugesh(self):
        print("Testing game against mrugesh...")
        actual = play(player, mrugesh, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat mrugesh at least 60% of the time.')


if __name__ == "__main__":
    unittest.main(argv=["first-arg-is-ignored"], exit=False)

....


Testing game against abbey...
Final results: {'p1': 521, 'p2': 273, 'tie': 206}
Player 1 win rate: 65.61712846347606%
Testing game against kris...
Final results: {'p1': 999, 'p2': 1, 'tie': 0}
Player 1 win rate: 99.9%
Testing game against mrugesh...
Final results: {'p1': 840, 'p2': 158, 'tie': 2}
Player 1 win rate: 84.16833667334669%
Testing game against quincy...
Final results: {'p1': 998, 'p2': 1, 'tie': 1}
Player 1 win rate: 99.8998998998999%



----------------------------------------------------------------------
Ran 4 tests in 0.018s

OK
